In [ ]:
#simple regression 1: Association between preventive measures and malaria history

import pandas as pd
import statsmodels.api as sm
import numpy as np

#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert outcome (malaria history) to binary
df['malaria_history'] = df['History of malaria in the past year in the household'].map({'Yes': 1, 'No': 0})

# Convert predictor (use of preventive measures) to binary
df['used_prevention'] = df['Use of malaria preventive measures'].map({'Yes': 1, 'No': 0})

# Set up predictor and outcome
X = sm.add_constant(df['used_prevention'])  # add intercept
y = df['malaria_history']

# Run logistic regression
model = sm.Logit(y, X).fit()

# Print regression results
print(model.summary())

# Calculate and display odds ratio + 95% confidence interval
odds_ratio = np.exp(model.params)
conf = np.exp(model.conf_int())
conf.columns = ['2.5%', '97.5%']

print("\nOdds Ratio and 95% CI:")
print(pd.concat([odds_ratio, conf], axis=1))


Optimization terminated successfully.
         Current function value: 0.663783
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        malaria_history   No. Observations:                  282
Model:                          Logit   Df Residuals:                      280
Method:                           MLE   Df Model:                            1
Date:                Sat, 10 May 2025   Pseudo R-squ.:               4.146e-06
Time:                        12:38:33   Log-Likelihood:                -187.19
converged:                       True   LL-Null:                       -187.19
Covariance Type:            nonrobust   LLR p-value:                    0.9686
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.4700      0.570      0.824      0.410      -0.647       1.587
used_preventio

In [ ]:
#simple regression 2: using cases of malaria as the outcome
# Filter out rows with missing or invalid data
df_clean = df[['Cases of malaria in the past year', 'used_prevention']].copy()
df_clean = df_clean.dropna()

# Ensure both columns are numeric
df_clean['Cases of malaria in the past year'] = pd.to_numeric(df_clean['Cases of malaria in the past year'], errors='coerce')
df_clean['used_prevention'] = pd.to_numeric(df_clean['used_prevention'], errors='coerce')

# Drop again if coercion introduced NaNs
df_clean = df_clean.dropna()

# Drop rows with negative malaria case counts (shouldn't happen, but just in case)
df_clean = df_clean[df_clean['Cases of malaria in the past year'] >= 0]


# Define X and y
X = sm.add_constant(df_clean['used_prevention'])
y = df_clean['Cases of malaria in the past year']

# Run Poisson model
poisson_model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# Show results
print(poisson_model.summary())

# IRRs
import numpy as np
irr = np.exp(poisson_model.params)
conf = np.exp(poisson_model.conf_int())
conf.columns = ['2.5%', '97.5%']

print("\nIRRs and 95% CI:")
print(pd.concat([irr, conf], axis=1))


                         Generalized Linear Model Regression Results                         
Dep. Variable:     Cases of malaria in the past year   No. Observations:                  281
Model:                                           GLM   Df Residuals:                      279
Model Family:                                Poisson   Df Model:                            1
Link Function:                                   Log   Scale:                          1.0000
Method:                                         IRLS   Log-Likelihood:                -562.35
Date:                               Sat, 10 May 2025   Deviance:                       650.84
Time:                                       14:28:08   Pearson chi2:                     683.
No. Iterations:                                    5   Pseudo R-squ. (CS):            0.02133
Covariance Type:                           nonrobust                                         
                      coef    std err          z      P>|z| 

In [ ]:
#separating the sample into rural and urban for preventive measures
import pandas as pd
import statsmodels.api as sm
import numpy as np

#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Map dependent and independent variables to numeric
data['History of malaria in the past year in the household'] = data['History of malaria in the past year in the household'].map({'Yes': 1, 'No': 0})
data['Use of malaria preventive measures'] = data['Use of malaria preventive measures'].map({'Yes': 1, 'No': 0})

# Filter rural and urban samples
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# Define a function to run logistic regression
def run_logistic(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    return result

# Prepare variables
predictor = 'Use of malaria preventive measures'
outcome = 'History of malaria in the past year in the household'

# 1. Rural regression
print("=== Rural Area Logistic Regression ===")
X_rural = rural_data[[predictor]]
y_rural = rural_data[outcome]

result_rural = run_logistic(X_rural, y_rural)
print(result_rural.summary())

odds_ratios_rural = pd.DataFrame({
    "OR": np.exp(result_rural.params),
    "2.5%": np.exp(result_rural.conf_int()[0]),
    "97.5%": np.exp(result_rural.conf_int()[1])
})
print("\nOdds Ratios and 95% CI (Rural):")
print(odds_ratios_rural)

# 2. Urban regression
print("\n=== Urban Area Logistic Regression ===")
X_urban = urban_data[[predictor]]
y_urban = urban_data[outcome]

result_urban = run_logistic(X_urban, y_urban)
print(result_urban.summary())

odds_ratios_urban = pd.DataFrame({
    "OR": np.exp(result_urban.params),
    "2.5%": np.exp(result_urban.conf_int()[0]),
    "97.5%": np.exp(result_urban.conf_int()[1])
})
print("\nOdds Ratios and 95% CI (Urban):")
print(odds_ratios_urban)

=== Rural Area Logistic Regression ===
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                   75
Model:                                                            Logit   Df Residuals:                       73
Method:                                                             MLE   Df Model:                            1
Date:                                                  Sun, 11 May 2025   Pseudo R-squ.:                 0.02546
Time:                                                          14:09:25   Log-Likelihood:                -37.886
converged:                                                        False   LL-Null:                       -38.875
Covariance Type:                                              nonrobust   LLR p-value:                    0.1594
                                         coef    std err 

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# for individual preventive measures


import pandas as pd
import statsmodels.api as sm
import numpy as np

# Load dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Clean column names (if needed)
data.columns = data.columns.str.strip()

# Transform outcome variable
data['History of malaria in the past year in the household'] = data['History of malaria in the past year in the household'].map({'Yes': 1, 'No': 0})

# Define preventive measure columns
preventive_measures = [
    'LLITNs',
    'Insecticide sprays',
    'Mosquito coils',
    'House modification e.g putting up wire nets on windows'
]

# Transform activities into ordinal values: Never = 0, Rarely = 1, Monthly/Weekly/Daily = 2
def transform_activity(x):
    if x == 'Never':
        return 0
    elif x == 'Rarely':
        return 1
    elif x in ['Monthly', 'Weekly', 'Daily']:
        return 2
    else:
        return np.nan

for col in preventive_measures:
    data[col] = data[col].apply(transform_activity)

# Function to run logistic regression
def run_logistic(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    return result

# Outcome variable
outcome = 'History of malaria in the past year in the household'

# Run separate logistic regression for each preventive measure
for measure in preventive_measures:
    print(f"\n=== Logistic Regression for {measure} ===")
    X = data[[measure]]
    y = data[outcome]

    # Drop rows with missing values
    df_temp = pd.concat([X, y], axis=1).dropna()
    X_clean = df_temp[[measure]]
    y_clean = df_temp[outcome]

    try:
        result = run_logistic(X_clean, y_clean)
        print(result.summary())

        odds_ratios = pd.DataFrame({
            "OR": np.exp(result.params),
            "2.5%": np.exp(result.conf_int()[0]),
            "97.5%": np.exp(result.conf_int()[1])
        })
        print("\nOdds Ratios and 95% CI:")
        print(odds_ratios)
    except Exception as e:
        print(f"⚠️ Regression failed for {measure}: {e}")


=== Logistic Regression for LLITNs ===
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  282
Model:                                                            Logit   Df Residuals:                      280
Method:                                                             MLE   Df Model:                            1
Date:                                                  Mon, 19 May 2025   Pseudo R-squ.:                 0.01276
Time:                                                          10:39:14   Log-Likelihood:                -184.80
converged:                                                         True   LL-Null:                       -187.19
Covariance Type:                                              nonrobust   LLR p-value:                   0.02886
                 coef    std err          z      P>|z|  

In [ ]:
#after separating sample into rural and urban having malaria history as the outcome

import pandas as pd
import statsmodels.api as sm
import numpy as np

# Load dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Clean column names
data.columns = data.columns.str.strip()

# Map outcome variable
data['History of malaria in the past year in the household'] = data['History of malaria in the past year in the household'].map({'Yes': 1, 'No': 0})

# Define preventive measures
preventive_measures = [
    'LLITNs',
    'Insecticide sprays',
    'Mosquito coils',
    'House modification e.g putting up wire nets on windows'
]

# Transform activities into ordinal values: Never = 0, Rarely = 1, Monthly/Weekly/Daily = 2
def transform_activity(x):
    if x == 'Never':
        return 0
    elif x == 'Rarely':
        return 1
    elif x in ['Monthly', 'Weekly', 'Daily']:
        return 2
    else:
        return np.nan

for col in preventive_measures:
    data[col] = data[col].apply(transform_activity)

# Logistic regression function
def run_logistic(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    return result

# Outcome variable
outcome = 'History of malaria in the past year in the household'

# Function to run models for a subset
def run_models_for_subset(subset_data, label="Full Sample"):
    print(f"\n====== Logistic Regressions for {label} ======")
    for measure in preventive_measures:
        print(f"\n--- Logistic Regression for {measure} ---")

        X = subset_data[[measure]]
        y = subset_data[outcome]

        df_temp = pd.concat([X, y], axis=1).dropna()
        X_clean = df_temp[[measure]]
        y_clean = df_temp[outcome]

        if X_clean.shape[0] > 0:  # Make sure there are observations left
            try:
                result = run_logistic(X_clean, y_clean)
                print(result.summary())

                odds_ratios = pd.DataFrame({
                    "OR": np.exp(result.params),
                    "2.5%": np.exp(result.conf_int()[0]),
                    "97.5%": np.exp(result.conf_int()[1])
                })
                print("\nOdds Ratios and 95% CI:")
                print(odds_ratios)
            except Exception as e:
                print(f"⚠️ Regression failed for {measure}: {e}")
        else:
            print(f"⚠️ No data available for {measure} in {label}")

# 1. Run models for the full dataset
run_models_for_subset(data, label="Full Sample")

# 2. Split the dataset
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# 3. Run models for Rural households
run_models_for_subset(rural_data, label="Rural Area")

# 4. Run models for Urban households
run_models_for_subset(urban_data, label="Urban Area")


====== Logistic Regressions for Full Sample ======

--- Logistic Regression for LLITNs ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  282
Model:                                                            Logit   Df Residuals:                      280
Method:                                                             MLE   Df Model:                            1
Date:                                                  Mon, 19 May 2025   Pseudo R-squ.:                 0.01276
Time:                                                          10:40:00   Log-Likelihood:                -184.80
converged:                                                         True   LL-Null:                       -187.19
Covariance Type:                                              nonrobust   LLR p-value:                   0.02886
    

In [ ]:
#for specific preventive measures and cases of malaria as the outcome

import pandas as pd
import statsmodels.api as sm
import numpy as np

# Load dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Clean column names
data.columns = data.columns.str.strip()

# Define preventive measures
preventive_measures = [
    'LLITNs',
    'Insecticide sprays',
    'Mosquito coils',
    'House modification e.g putting up wire nets on windows'
]

# Transform activities into ordinal values: Never = 0, Rarely = 1, Monthly/Weekly/Daily = 2
def transform_activity(x):
    if x == 'Never':
        return 0
    elif x == 'Rarely':
        return 1
    elif x in ['Monthly', 'Weekly', 'Daily']:
        return 2
    else:
        return np.nan

for col in preventive_measures:
    data[col] = data[col].apply(transform_activity)

# Transform "Cases of malaria in the past year"
# Replace cases > 10 with the household size
data['Cases of malaria in the past year'] = np.where(
    data['Cases of malaria in the past year'] > 10,
    data['Household size'],
    data['Cases of malaria in the past year']
)

# Function to run Poisson regression
def run_poisson(X, y):
    X = sm.add_constant(X)
    model = sm.Poisson(y, X)
    result = model.fit(disp=0)
    return result

# Outcome variable
outcome = 'Cases of malaria in the past year'

# Function to run models for a subset
def run_poisson_models_for_subset(subset_data, label="Full Sample"):
    print(f"\n====== Poisson Regressions for {label} ======")
    for measure in preventive_measures:
        print(f"\n--- Poisson Regression for {measure} ---")

        X = subset_data[[measure]]
        y = subset_data[outcome]

        df_temp = pd.concat([X, y], axis=1).dropna()
        X_clean = df_temp[[measure]]
        y_clean = df_temp[outcome]

        if X_clean.shape[0] > 0:  # Make sure there are observations left
            try:
                result = run_poisson(X_clean, y_clean)
                print(result.summary())

                incidence_rate_ratios = pd.DataFrame({
                    "IRR": np.exp(result.params),
                    "2.5%": np.exp(result.conf_int()[0]),
                    "97.5%": np.exp(result.conf_int()[1])
                })
                print("\nIncidence Rate Ratios (IRR) and 95% CI:")
                print(incidence_rate_ratios)
            except Exception as e:
                print(f"⚠️ Regression failed for {measure}: {e}")
        else:
            print(f"⚠️ No data available for {measure} in {label}")

# 1. Run models for the full dataset
run_poisson_models_for_subset(data, label="Full Sample")

# 2. Split the dataset
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# 3. Run models for Rural households
run_poisson_models_for_subset(rural_data, label="Rural Area")

# 4. Run models for Urban households
run_poisson_models_for_subset(urban_data, label="Urban Area")


====== Poisson Regressions for Full Sample ======

--- Poisson Regression for LLITNs ---
                                  Poisson Regression Results                                 
Dep. Variable:     Cases of malaria in the past year   No. Observations:                  282
Model:                                       Poisson   Df Residuals:                      280
Method:                                          MLE   Df Model:                            1
Date:                               Mon, 19 May 2025   Pseudo R-squ.:                0.001266
Time:                                       10:42:42   Log-Likelihood:                -566.35
converged:                                      True   LL-Null:                       -567.06
Covariance Type:                           nonrobust   LLR p-value:                    0.2308
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------


In [ ]:
#for household measures (first binary, then specific ones) as predictors and history of malaria and cases of malaria as the outcome

import pandas as pd
import statsmodels.api as sm
import numpy as np

# Load dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Clean column names: remove extra spaces but KEEP normal spaces
data.columns = data.columns.str.strip()

# Recode outcome: History of malaria
data['History of malaria in the past year in the household'] = data['History of malaria in the past year in the household'].map({'Yes': 1, 'No': 0})

# Transform 'Cases of malaria' > 10 to Household size
data['Cases of malaria in the past year'] = np.where(
    data['Cases of malaria in the past year'] > 10,
    data['Household size'],
    data['Cases of malaria in the past year']
)

# Recode main environmental engagement variable
data['Engaged in household environmental measures'] = data['Engaged in household environmental measures'].map({'Yes': 1, 'No': 0})

# Define individual environmental practices (your exact names)
environmental_practices = [
    'Eliminating standing water',
    'Managing trash cans',
    'Mowing grass around the house',
    'Maintaining drainage systems used to direct water away from homes',
    'Covering water storage containers',
    'Sweeping the yard'
]

# Transform activities into ordinal values: Never = 0, Rarely = 1, Monthly/Weekly/Daily = 2
def transform_activity(x):
    if x == 'Never':
        return 0
    elif x == 'Rarely':
        return 1
    elif x in ['Monthly', 'Weekly', 'Daily']:
        return 2
    else:
        return np.nan

for col in environmental_practices:
    data[col] = data[col].apply(transform_activity)

# Logistic regression function
def run_logistic(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    return result

# Poisson regression function
def run_poisson(X, y):
    X = sm.add_constant(X)
    model = sm.Poisson(y, X)
    result = model.fit(disp=0)
    return result

# Function to run models for a subset
def run_models(subset_data, predictor_list, outcome, model_type="logistic", label="Full Sample"):
    print(f"\n====== {model_type.capitalize()} Regressions for {label} (Outcome: {outcome}) ======")
    for predictor in predictor_list:
        print(f"\n--- Regression for Predictor: {predictor} ---")

        X = subset_data[[predictor]]
        y = subset_data[outcome]

        df_temp = pd.concat([X, y], axis=1).dropna()
        X_clean = df_temp[[predictor]]
        y_clean = df_temp[outcome]

        if X_clean.shape[0] > 0:
            try:
                if model_type == "logistic":
                    result = run_logistic(X_clean, y_clean)
                else:
                    result = run_poisson(X_clean, y_clean)

                print(result.summary())

                if model_type == "logistic":
                    or_table = pd.DataFrame({
                        "OR": np.exp(result.params),
                        "2.5%": np.exp(result.conf_int()[0]),
                        "97.5%": np.exp(result.conf_int()[1])
                    })
                    print("\nOdds Ratios (OR) and 95% CI:")
                    print(or_table)
                else:
                    irr_table = pd.DataFrame({
                        "IRR": np.exp(result.params),
                        "2.5%": np.exp(result.conf_int()[0]),
                        "97.5%": np.exp(result.conf_int()[1])
                    })
                    print("\nIncidence Rate Ratios (IRR) and 95% CI:")
                    print(irr_table)

            except Exception as e:
                print(f"⚠️ Regression failed for {predictor}: {e}")
        else:
            print(f"⚠️ No data for {predictor} in {label}")

# Main predictor
main_predictor = ['Engaged in household environmental measures']

# Correct outcome names
outcome_logistic = 'History of malaria in the past year in the household'
outcome_poisson = 'Cases of malaria in the past year'

# Split data into Rural and Urban
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# ===== RUN ALL MODELS =====

# Full sample
run_models(data, main_predictor, outcome=outcome_logistic, model_type="logistic", label="Full Sample")
run_models(data, environmental_practices, outcome=outcome_logistic, model_type="logistic", label="Full Sample")
run_models(data, main_predictor, outcome=outcome_poisson, model_type="poisson", label="Full Sample")
run_models(data, environmental_practices, outcome=outcome_poisson, model_type="poisson", label="Full Sample")

# Rural sample
run_models(rural_data, main_predictor, outcome=outcome_logistic, model_type="logistic", label="Rural Area")
run_models(rural_data, environmental_practices, outcome=outcome_logistic, model_type="logistic", label="Rural Area")
run_models(rural_data, main_predictor, outcome=outcome_poisson, model_type="poisson", label="Rural Area")
run_models(rural_data, environmental_practices, outcome=outcome_poisson, model_type="poisson", label="Rural Area")

# Urban sample
run_models(urban_data, main_predictor, outcome=outcome_logistic, model_type="logistic", label="Urban Area")
run_models(urban_data, environmental_practices, outcome=outcome_logistic, model_type="logistic", label="Urban Area")
run_models(urban_data, main_predictor, outcome=outcome_poisson, model_type="poisson", label="Urban Area")
run_models(urban_data, environmental_practices, outcome=outcome_poisson, model_type="poisson", label="Urban Area")


====== Logistic Regressions for Full Sample (Outcome: History of malaria in the past year in the household) ======

--- Regression for Predictor: Engaged in household environmental measures ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  282
Model:                                                            Logit   Df Residuals:                      280
Method:                                                             MLE   Df Model:                            1
Date:                                                  Mon, 19 May 2025   Pseudo R-squ.:               0.0006384
Time:                                                          10:44:43   Log-Likelihood:                -187.07
converged:                                                         True   LL-Null:                       -187.19
Covariance Typ

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                   75
Model:                                                            Logit   Df Residuals:                       73
Method:                                                             MLE   Df Model:                            1
Date:                                                  Mon, 19 May 2025   Pseudo R-squ.:                 0.03209
Time:                                                          10:44:43   Log-Likelihood:                -37.628
converged:                                                        False   LL-Null:                       -38.875
Covariance Type:                                              nonrobust   LLR p-value:                    0.1142
                                                  coef    std err          z      P>|z|      [0.

In [ ]:
#engaged in household practices as predictor and breeding sites as outcome
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Load dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Clean column names
data.columns = data.columns.str.strip()

# Transform outcome: 'Observed breeding sites in the past year' to binary
data['Observed breeding sites in the past year'] = data['Observed breeding sites in the past year'].map({'Yes': 1, 'No': 0})

# Recode main environmental engagement variable
data['Engaged in household environmental measures'] = data['Engaged in household environmental measures'].map({'Yes': 1, 'No': 0})

# Define individual environmental practices (your exact names)
environmental_practices = [
    'Eliminating standing water',
    'Managing trash cans',
    'Mowing grass around the house',
    'Maintaining drainage systems used to direct water away from homes',
    'Covering water storage containers',
    'Sweeping the yard'
]

# Transform activities into ordinal values: Never = 0, Rarely = 1, Monthly/Weekly/Daily = 2
def transform_activity(x):
    if x == 'Never':
        return 0
    elif x == 'Rarely':
        return 1
    elif x in ['Monthly', 'Weekly', 'Daily']:
        return 2
    else:
        return np.nan

for col in environmental_practices:
    data[col] = data[col].apply(transform_activity)

# Logistic regression function
def run_logistic(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    return result

# General function to run regression
def simple_regression(subset_data, predictor_list, outcome, model_type="logistic", label="Full Sample"):
    print(f"\n====== {model_type.capitalize()} Regressions for {label} (Outcome: {outcome}) ======")
    for predictor in predictor_list:
        print(f"\n--- Regression for Predictor: {predictor} ---")

        X = subset_data[[predictor]]
        y = subset_data[outcome]

        df_temp = pd.concat([X, y], axis=1).dropna()
        X_clean = df_temp[[predictor]]
        y_clean = df_temp[outcome]

        if X_clean.shape[0] > 0:
            try:
                result = run_logistic(X_clean, y_clean)

                print(result.summary())

                or_table = pd.DataFrame({
                    "OR": np.exp(result.params),
                    "2.5%": np.exp(result.conf_int()[0]),
                    "97.5%": np.exp(result.conf_int()[1])
                })
                print("\nOdds Ratios (OR) and 95% CI:")
                print(or_table)

            except Exception as e:
                print(f"⚠️ Regression failed for {predictor}: {e}")
        else:
            print(f"⚠️ No valid data for {predictor} in {label}")

# Main predictor
main_predictor = ['Engaged in household environmental measures']

# Outcome
outcome_logistic = 'Observed breeding sites in the past year'

# Split data
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# ===== RUNNING =====

# 1. Full Sample - General
simple_regression(data, main_predictor, outcome=outcome_logistic, model_type="logistic", label="Full Sample")

# 2. Full Sample - Individual practices
simple_regression(data, environmental_practices, outcome=outcome_logistic, model_type="logistic", label="Full Sample")

# 3. Rural - General
simple_regression(rural_data, main_predictor, outcome=outcome_logistic, model_type="logistic", label="Rural Area")

# 4. Rural - Individual practices
simple_regression(rural_data, environmental_practices, outcome=outcome_logistic, model_type="logistic", label="Rural Area")

# 5. Urban - General
simple_regression(urban_data, main_predictor, outcome=outcome_logistic, model_type="logistic", label="Urban Area")

# 6. Urban - Individual practices
simple_regression(urban_data, environmental_practices, outcome=outcome_logistic, model_type="logistic", label="Urban Area")


====== Logistic Regressions for Full Sample (Outcome: Observed breeding sites in the past year) ======

--- Regression for Predictor: Engaged in household environmental measures ---
                                      Logit Regression Results                                      
Dep. Variable:     Observed breeding sites in the past year   No. Observations:                  282
Model:                                                Logit   Df Residuals:                      280
Method:                                                 MLE   Df Model:                            1
Date:                                      Mon, 19 May 2025   Pseudo R-squ.:                 0.01273
Time:                                              10:51:24   Log-Likelihood:                -133.08
converged:                                             True   LL-Null:                       -134.79
Covariance Type:                                  nonrobust   LLR p-value:                   0.06399
         

In [ ]:
#community engagement vs malaria history and cases of malaria as well as breeding sites
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Load dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Step 2: Clean column names
data.columns = data.columns.str.strip()

# Step 3: Transform outcome variables
data['Observed breeding sites in the past year'] = data['Observed breeding sites in the past year'].map({'Yes': 1, 'No': 0})

# Cap malaria cases at household size if >10
data['Cases of malaria in the past year'] = np.where(
    data['Cases of malaria in the past year'] > 10,
    data['Household size'],
    data['Cases of malaria in the past year']
)

# Main community engagement variable
data['Community engagement in control of mosquito breeding sites'] = data['Community engagement in control of mosquito breeding sites'].map({'Yes': 1, 'No': 0})

# Binary variable for malaria history
data['History of malaria in the past year in the household'] = data['History of malaria in the past year in the household'].map({'Yes': 1, 'No': 0})

# Step 4: Transform activity-level variables
community_activities = [
    'Removing stagnant water in open places like markets and hospitals',
    'Mowing grass in open places',
    'Filling potholes',
    'Sweeping around markets',
    'Information sharing'
]

def transform_activity(x):
    if x == 'Never':
        return 0
    elif x == 'Rarely':
        return 1
    elif x in ['Monthly', 'Weekly', 'Daily']:
        return 2
    else:
        return np.nan

for col in community_activities:
    data[col] = data[col].apply(transform_activity)

# Step 5: Regression functions
def run_logistic(X, y):
    X = sm.add_constant(X)
    model = sm.Logit(y, X)
    result = model.fit(disp=0)
    return result

def run_poisson(X, y):
    X = sm.add_constant(X)
    model = sm.Poisson(y, X)
    result = model.fit(disp=0)
    return result

def simple_regression(subset_data, predictor_list, outcome, model_type="logistic", label="Full Sample"):
    print(f"\n====== {model_type.capitalize()} Regressions for {label} (Outcome: {outcome}) ======")
    for predictor in predictor_list:
        print(f"\n--- Regression for Predictor: {predictor} ---")

        X = subset_data[[predictor]]
        y = subset_data[outcome]

        df_temp = pd.concat([X, y], axis=1).dropna()
        X_clean = df_temp[[predictor]]
        y_clean = df_temp[outcome]

        if X_clean.shape[0] > 0:
            try:
                if model_type == "logistic":
                    result = run_logistic(X_clean, y_clean)
                else:
                    result = run_poisson(X_clean, y_clean)

                print(result.summary())

                if model_type == "logistic":
                    or_table = pd.DataFrame({
                        "OR": np.exp(result.params),
                        "2.5%": np.exp(result.conf_int()[0]),
                        "97.5%": np.exp(result.conf_int()[1])
                    })
                    print("\nOdds Ratios (OR) and 95% CI:")
                    print(or_table)
                else:
                    irr_table = pd.DataFrame({
                        "IRR": np.exp(result.params),
                        "2.5%": np.exp(result.conf_int()[0]),
                        "97.5%": np.exp(result.conf_int()[1])
                    })
                    print("\nIncidence Rate Ratios (IRR) and 95% CI:")
                    print(irr_table)

            except Exception as e:
                print(f"⚠️ Regression failed for {predictor}: {e}")
        else:
            print(f"⚠️ No valid data for {predictor} in {label}")

# Step 6: Define predictors and outcomes
main_community_engagement = ['Community engagement in control of mosquito breeding sites']
outcome_poisson = 'Cases of malaria in the past year'
outcome_logistic_breeding = 'Observed breeding sites in the past year'
outcome_logistic_history = 'History of malaria in the past year in the household'

# Step 7: Split by residence area
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# ===== RUNNING ANALYSIS =====

# --- Poisson regression: Cases of malaria ---
simple_regression(data, main_community_engagement, outcome=outcome_poisson, model_type="poisson", label="Full Sample")
simple_regression(data, community_activities, outcome=outcome_poisson, model_type="poisson", label="Full Sample")

simple_regression(rural_data, main_community_engagement, outcome=outcome_poisson, model_type="poisson", label="Rural Area")
simple_regression(rural_data, community_activities, outcome=outcome_poisson, model_type="poisson", label="Rural Area")

simple_regression(urban_data, main_community_engagement, outcome=outcome_poisson, model_type="poisson", label="Urban Area")
simple_regression(urban_data, community_activities, outcome=outcome_poisson, model_type="poisson", label="Urban Area")

# --- Logistic regression: Observed breeding sites ---
simple_regression(data, main_community_engagement, outcome=outcome_logistic_breeding, model_type="logistic", label="Full Sample")
simple_regression(data, community_activities, outcome=outcome_logistic_breeding, model_type="logistic", label="Full Sample")

simple_regression(rural_data, main_community_engagement, outcome=outcome_logistic_breeding, model_type="logistic", label="Rural Area")
simple_regression(rural_data, community_activities, outcome=outcome_logistic_breeding, model_type="logistic", label="Rural Area")

simple_regression(urban_data, main_community_engagement, outcome=outcome_logistic_breeding, model_type="logistic", label="Urban Area")
simple_regression(urban_data, community_activities, outcome=outcome_logistic_breeding, model_type="logistic", label="Urban Area")

# --- Logistic regression: History of malaria in household ---
simple_regression(data, main_community_engagement, outcome=outcome_logistic_history, model_type="logistic", label="Full Sample")
simple_regression(data, community_activities, outcome=outcome_logistic_history, model_type="logistic", label="Full Sample")

simple_regression(rural_data, main_community_engagement, outcome=outcome_logistic_history, model_type="logistic", label="Rural Area")
simple_regression(rural_data, community_activities, outcome=outcome_logistic_history, model_type="logistic", label="Rural Area")

simple_regression(urban_data, main_community_engagement, outcome=outcome_logistic_history, model_type="logistic", label="Urban Area")
simple_regression(urban_data, community_activities, outcome=outcome_logistic_history, model_type="logistic", label="Urban Area")


====== Poisson Regressions for Full Sample (Outcome: Cases of malaria in the past year) ======

--- Regression for Predictor: Community engagement in control of mosquito breeding sites ---
                                  Poisson Regression Results                                 
Dep. Variable:     Cases of malaria in the past year   No. Observations:                  282
Model:                                       Poisson   Df Residuals:                      280
Method:                                          MLE   Df Model:                            1
Date:                               Wed, 21 May 2025   Pseudo R-squ.:                0.003557
Time:                                       17:34:39   Log-Likelihood:                -565.05
converged:                                      True   LL-Null:                       -567.06
Covariance Type:                           nonrobust   LLR p-value:                   0.04459
                                                          

In [ ]:
#hierarchical regression with control variables (preventive measures vs malaria history)

import pandas as pd
import statsmodels.api as sm
import numpy as np

#Load your dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Map binary and ordinal variables
binary_maps = {
    'History of malaria in the past year in the household': {'Yes': 1, 'No': 0},
    'Use of malaria preventive measures': {'Yes': 1, 'No': 0},
    'Gender': {'Male': 1, 'Female': 0},
    'Level of education': {'Higher education': 1, 'Secondary level': 0},
    'Occupation': {'Employed': 0, 'Unemployed': 1}
}

ordinal_maps = {
    'Marital status': {'Divorced': 0, 'Single': 1, 'Married': 2},
    'Length of stay in area of residence': {
        'Less than 1 year': 0,
        '1-5 years': 1,
        'More than 5 years': 2
    }
}

# Apply mappings
for col, mapping in binary_maps.items():
    data[col] = data[col].map(mapping)

for col, mapping in ordinal_maps.items():
    data[col] = data[col].map(mapping)

# Convert to numeric
data['Household size'] = pd.to_numeric(data['Household size'], errors='coerce')

# Drop rows with missing values in any relevant columns
data = data.dropna(subset=[
    'History of malaria in the past year in the household',
    'Use of malaria preventive measures',
    'Age', 'Gender', 'Level of education', 'Marital status',
    'Occupation', 'Household size', 'Length of stay in area of residence',
    'Area of residence'
])

# Split rural and urban data
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# Logistic regression function with more iterations and error handling
def run_logistic(X, y):
    X = sm.add_constant(X, has_constant='add')
    try:
        model = sm.Logit(y, X)
        result = model.fit(disp=0, maxiter=100)
        return result
    except Exception as e:
        raise RuntimeError(f"Fit failed: {str(e)}")

# Define predictor and outcome
predictor = 'Use of malaria preventive measures'
outcome = 'History of malaria in the past year in the household'

# Define control variable sets
control_vars = [
    ['Age'],
    ['Age', 'Gender'],
    ['Age', 'Gender', 'Level of education'],
    ['Age', 'Gender', 'Level of education', 'Marital status'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence', 'Area of residence']

]

# Hierarchical logistic regression for rural/urban data
def hierarchical_logistic(area_data, area_name):
    print(f"\n=== {area_name} Logistic Regression (Hierarchical) ===")
    for i, controls in enumerate(control_vars):
        model_vars = [predictor] + controls
        X = area_data[model_vars]
        y = area_data[outcome]
        print(f"\n--- Model {i+1}: Controls: {', '.join(controls)} ---")

        try:
            result = run_logistic(X, y)
            print(result.summary())
            odds_ratios = pd.DataFrame({
                'OR': np.exp(result.params),
                '2.5%': np.exp(result.conf_int()[0]),
                '97.5%': np.exp(result.conf_int()[1])
            })
            print("\nOdds Ratios with 95% CI:")
            print(odds_ratios)
        except Exception as e:
            print(f"Model {i+1} failed: {e}")

# Run hierarchical logistic regression
hierarchical_logistic(rural_data, "Rural Area")
hierarchical_logistic(urban_data, "Urban Area")


=== Rural Area Logistic Regression (Hierarchical) ===

--- Model 1: Controls: Age ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                   75
Model:                                                            Logit   Df Residuals:                       72
Method:                                                             MLE   Df Model:                            2
Date:                                                  Wed, 14 May 2025   Pseudo R-squ.:                 0.03147
Time:                                                          11:28:53   Log-Likelihood:                -37.652
converged:                                                        False   LL-Null:                       -38.875
Covariance Type:                                              nonrobust   LLR p-value:                    0.2942
         

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.11/dist-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/

                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  207
Model:                                                            Logit   Df Residuals:                      202
Method:                                                             MLE   Df Model:                            4
Date:                                                  Wed, 14 May 2025   Pseudo R-squ.:                 0.02644
Time:                                                          11:28:54   Log-Likelihood:                -138.21
converged:                                                         True   LL-Null:                       -141.97
Covariance Type:                                              nonrobust   LLR p-value:                    0.1114
                                         coef    std err          z      P>|z|      [0.025      

In [ ]:
import pandas as pd

# Load your dataset
#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# === 1. Descriptive statistics for household size ===
print("=== Descriptive Statistics: Household Size ===")
household_stats = data['Household size'].describe()
print(f"Mean: {household_stats['mean']:.2f}")
print(f"Median: {data['Household size'].median():.2f}")
print(f"Standard Deviation: {data['Household size'].std():.2f}")


# === 2. Frequency tables for household environmental practices ===
practice_columns = [
    'Eliminating standing water',
    'Maintaining drainage systems used to direct water away from homes',
    'Mowing grass around the house',
    'Covering water storage containers',
    'Managing trash cans',
    'Sweeping the yard',
    'Frequency of other household environmental measures not included in the survey'
]

# Rename last variable to "other"
column_mapping = {
    'Frequency of other household environmental measures not included in the survey': 'Other'
}

data_renamed = data.rename(columns=column_mapping)

# Update column list to reflect renaming
practice_columns = [
    'Eliminating standing water',
    'Maintaining drainage systems used to direct water away from homes',
    'Mowing grass around the house',
    'Covering water storage containers',
    'Managing trash cans',
    'Sweeping the yard',
    'Other'
]

# Define a function to map categories
def simplify_freq(val):
    if pd.isna(val):
        return 'No Response'
    val = val.strip().lower()
    if val in ['daily', 'weekly']:
        return 'Weekly/Daily'
    elif val in ['monthly', 'rarely']:
        return 'Rarely/Monthly'
    elif val == 'never':
        return 'Never'
    else:
        return 'Other'

print("\n=== Frequency of Household Environmental Practices (with Percentages) ===")
for col in practice_columns:
    simplified = data_renamed[col].apply(simplify_freq)
    freq = simplified.value_counts(dropna=False)
    perc = simplified.value_counts(normalize=True, dropna=False) * 100
    combined = pd.concat([freq, perc], axis=1)
    combined.columns = ['Count', 'Percentage (%)']
    print(f"\n{col}:")
    print(combined.round(2))

=== Descriptive Statistics: Household Size ===
Mean: 4.59
Median: 4.00
Standard Deviation: 2.29

=== Frequency of Household Environmental Practices (with Percentages) ===

Eliminating standing water:
                            Count  Percentage (%)
Eliminating standing water                       
Rarely/Monthly                120           42.55
Never                          92           32.62
Weekly/Daily                   70           24.82

Maintaining drainage systems used to direct water away from homes:
                                                    Count  Percentage (%)
Maintaining drainage systems used to direct wat...                       
Rarely/Monthly                                        126           44.68
Never                                                 108           38.30
Weekly/Daily                                           48           17.02

Mowing grass around the house:
                               Count  Percentage (%)
Mowing grass around the hou

In [ ]:
#hierarchical regression between malaria preventive measures and history of malaria (introducing controls one by one)

import pandas as pd
import statsmodels.api as sm
import numpy as np

#step 1 Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)


# Step 2: Map binary and ordinal variables
binary_maps = {
    'History of malaria in the past year in the household': {'Yes': 1, 'No': 0},
    'Use of malaria preventive measures': {'Yes': 1, 'No': 0},
    'Gender': {'Male': 1, 'Female': 0},
    'Level of education': {'Higher education': 1, 'Secondary level': 0},
    'Occupation': {'Employed': 0, 'Unemployed': 1}
}

ordinal_maps = {
    'Marital status': {'Divorced': 0, 'Single': 1, 'Married': 2},
    'Length of stay in area of residence': {
        'Less than 1 year': 0,
        '1-5 years': 1,
        'More than 5 years': 2
    }
}

# Apply mappings
for col, mapping in binary_maps.items():
    data[col] = data[col].map(mapping)

for col, mapping in ordinal_maps.items():
    data[col] = data[col].map(mapping)

data['Household size'] = pd.to_numeric(data['Household size'], errors='coerce')

# Drop rows with missing values
data = data.dropna(subset=[
    'History of malaria in the past year in the household',
    'Use of malaria preventive measures',
    'Age', 'Gender', 'Level of education', 'Marital status',
    'Occupation', 'Household size', 'Length of stay in area of residence',
    'Area of residence'
])

# Split rural and urban data
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# Logistic regression function
def run_logistic(X, y):
    X = sm.add_constant(X, has_constant='add')
    try:
        model = sm.Logit(y, X)
        result = model.fit(disp=0, maxiter=100)
        return result
    except Exception as e:
        raise RuntimeError(f"Fit failed: {str(e)}")

# Variables
predictor = 'Use of malaria preventive measures'
outcome = 'History of malaria in the past year in the household'

# Control variable sets for whole sample (includes area of residence)
full_controls = [
    ['Age'],
    ['Age', 'Gender'],
    ['Age', 'Gender', 'Level of education'],
    ['Age', 'Gender', 'Level of education', 'Marital status'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence', 'Area of residence']
]

# Control sets for subgroup (area of residence removed)
subgroup_controls = [
    ['Age'],
    ['Age', 'Gender'],
    ['Age', 'Gender', 'Level of education'],
    ['Age', 'Gender', 'Level of education', 'Marital status'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence']
]

# Function for hierarchical logistic regression
def hierarchical_logistic(data_subset, area_name, controls_list):
    print(f"\n=== {area_name} Logistic Regression (Hierarchical) ===")
    for i, controls in enumerate(controls_list):
        model_vars = [predictor] + controls
        X = data_subset[model_vars]
        y = data_subset[outcome]
        print(f"\n--- Model {i+1}: Controls: {', '.join(controls)} ---")

        try:
            result = run_logistic(X, y)
            print(result.summary())
            odds_ratios = pd.DataFrame({
                'OR': np.exp(result.params),
                '2.5%': np.exp(result.conf_int()[0]),
                '97.5%': np.exp(result.conf_int()[1])
            })
            print("\nOdds Ratios with 95% CI:")
            print(odds_ratios)
        except Exception as e:
            print(f"Model {i+1} failed: {e}")

# Run hierarchical logistic regressions
hierarchical_logistic(data, "Whole Sample", full_controls)
hierarchical_logistic(rural_data, "Rural Area", subgroup_controls)
hierarchical_logistic(urban_data, "Urban Area", subgroup_controls)


=== Whole Sample Logistic Regression (Hierarchical) ===

--- Model 1: Controls: Age ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  282
Model:                                                            Logit   Df Residuals:                      279
Method:                                                             MLE   Df Model:                            2
Date:                                                  Mon, 19 May 2025   Pseudo R-squ.:               7.166e-05
Time:                                                          11:33:38   Log-Likelihood:                -187.17
converged:                                                         True   LL-Null:                       -187.19
Covariance Type:                                              nonrobust   LLR p-value:                    0.9867
       

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.11/dist-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/

Model 5 failed: Fit failed: Singular matrix

--- Model 6: Controls: Age, Gender, Level of education, Marital status, Occupation, Household size ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                   75
Model:                                                            Logit   Df Residuals:                       67
Method:                                                             MLE   Df Model:                            7
Date:                                                  Mon, 19 May 2025   Pseudo R-squ.:                  0.2001
Time:                                                          11:33:38   Log-Likelihood:                -31.095
converged:                                                        False   LL-Null:                       -38.875
Covariance Type:                                             

In [ ]:
#hierarchical with environmental practices as the predictor and malaria hist as outcome

import pandas as pd
import statsmodels.api as sm
import numpy as np

# Step 1: Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Step 2: Map binary and ordinal variables
binary_maps = {
    'History of malaria in the past year in the household': {'Yes': 1, 'No': 0},
    'Engaged in household environmental measures': {'Yes': 1, 'No': 0},  # <-- updated predictor
    'Gender': {'Male': 1, 'Female': 0},
    'Level of education': {'Higher education': 1, 'Secondary level': 0},
    'Occupation': {'Employed': 0, 'Unemployed': 1}
}

ordinal_maps = {
    'Marital status': {'Divorced': 0, 'Single': 1, 'Married': 2},
    'Length of stay in area of residence': {
        'Less than 1 year': 0,
        '1-5 years': 1,
        'More than 5 years': 2
    }
}

# Apply mappings
for col, mapping in binary_maps.items():
    data[col] = data[col].map(mapping)

for col, mapping in ordinal_maps.items():
    data[col] = data[col].map(mapping)

# Convert numeric
data['Household size'] = pd.to_numeric(data['Household size'], errors='coerce')

# Drop rows with missing values in key columns
data = data.dropna(subset=[
    'History of malaria in the past year in the household',
    'Engaged in household environmental measures',
    'Age', 'Gender', 'Level of education', 'Marital status',
    'Occupation', 'Household size', 'Length of stay in area of residence',
    'Area of residence'
])

# Split rural and urban data
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# Logistic regression helper
def run_logistic(X, y):
    X = sm.add_constant(X, has_constant='add')
    try:
        model = sm.Logit(y, X)
        result = model.fit(disp=0, maxiter=100)
        return result
    except Exception as e:
        raise RuntimeError(f"Fit failed: {str(e)}")

# Set outcome and predictor
predictor = 'Engaged in household environmental measures'
outcome = 'History of malaria in the past year in the household'

# Control variable sets (includes Area of residence for full sample only)
control_vars_full = [
    ['Age'],
    ['Age', 'Gender'],
    ['Age', 'Gender', 'Level of education'],
    ['Age', 'Gender', 'Level of education', 'Marital status'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence', 'Area of residence']
]

# For rural/urban samples (no Area of residence)
control_vars_split = [vars[:-1] for vars in control_vars_full if 'Area of residence' in vars or len(vars) < 8]

# Hierarchical logistic regression function
def hierarchical_logistic(area_data, area_name, control_vars):
    print(f"\n=== {area_name} Logistic Regression (Hierarchical) ===")
    for i, controls in enumerate(control_vars):
        model_vars = [predictor] + controls
        X = area_data[model_vars]
        y = area_data[outcome]
        print(f"\n--- Model {i+1}: Controls: {', '.join(controls)} ---")
        try:
            result = run_logistic(X, y)
            print(result.summary())
            odds_ratios = pd.DataFrame({
                'OR': np.exp(result.params),
                '2.5%': np.exp(result.conf_int()[0]),
                '97.5%': np.exp(result.conf_int()[1])
            })
            print("\nOdds Ratios with 95% CI:")
            print(odds_ratios)
        except Exception as e:
            print(f"Model {i+1} failed: {e}")

# Run regression for full dataset
hierarchical_logistic(data, "Full Sample", control_vars_full)

# Run regression for rural and urban (excluding 'Area of residence' as a control)
hierarchical_logistic(rural_data, "Rural Area", control_vars_split)
hierarchical_logistic(urban_data, "Urban Area", control_vars_split)


=== Full Sample Logistic Regression (Hierarchical) ===

--- Model 1: Controls: Age ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  282
Model:                                                            Logit   Df Residuals:                      279
Method:                                                             MLE   Df Model:                            2
Date:                                                  Tue, 20 May 2025   Pseudo R-squ.:               0.0007127
Time:                                                          09:03:55   Log-Likelihood:                -187.05
converged:                                                         True   LL-Null:                       -187.19
Covariance Type:                                              nonrobust   LLR p-value:                    0.8751
        

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/statsmodels/bas

                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                   75
Model:                                                            Logit   Df Residuals:                       71
Method:                                                             MLE   Df Model:                            3
Date:                                                  Tue, 20 May 2025   Pseudo R-squ.:                  0.1443
Time:                                                          09:03:56   Log-Likelihood:                -33.264
converged:                                                        False   LL-Null:                       -38.875
Covariance Type:                                              nonrobust   LLR p-value:                   0.01058
                                                  coef    std err          z      P>|z|      [0.

In [ ]:
#hierachical with community eng as the pred and malaria his as the outcome

import pandas as pd
import statsmodels.api as sm
import numpy as np

 #Step 1: Load the data
file_path = r'/content/Data of Malaria (THESIS) 17 april.xlsx'
sheet_name = 'Form responses 1'

#step 2 read the data
data = pd.read_excel(file_path, sheet_name=sheet_name)

# Step 2: Map binary and ordinal variables
binary_maps = {
    'History of malaria in the past year in the household': {'Yes': 1, 'No': 0},
    'Community engagement in control of mosquito breeding sites': {'Yes': 1, 'No': 0},  # <-- updated predictor
    'Gender': {'Male': 1, 'Female': 0},
    'Level of education': {'Higher education': 1, 'Secondary level': 0},
    'Occupation': {'Employed': 0, 'Unemployed': 1}
}

ordinal_maps = {
    'Marital status': {'Divorced': 0, 'Single': 1, 'Married': 2},
    'Length of stay in area of residence': {
        'Less than 1 year': 0,
        '1-5 years': 1,
        'More than 5 years': 2
    }
}

# Apply mappings
for col, mapping in binary_maps.items():
    data[col] = data[col].map(mapping)

for col, mapping in ordinal_maps.items():
    data[col] = data[col].map(mapping)

# Convert numeric
data['Household size'] = pd.to_numeric(data['Household size'], errors='coerce')

# Drop missing
data = data.dropna(subset=[
    'History of malaria in the past year in the household',
    'Community engagement in control of mosquito breeding sites',
    'Age', 'Gender', 'Level of education', 'Marital status',
    'Occupation', 'Household size', 'Length of stay in area of residence',
    'Area of residence'
])

# Split rural and urban data
rural_data = data[data['Area of residence'] == 'Rural area']
urban_data = data[data['Area of residence'] == 'Urban area']

# Logistic regression helper
def run_logistic(X, y):
    X = sm.add_constant(X, has_constant='add')
    try:
        model = sm.Logit(y, X)
        result = model.fit(disp=0, maxiter=100)
        return result
    except Exception as e:
        raise RuntimeError(f"Fit failed: {str(e)}")

# Set outcome and predictor
predictor = 'Community engagement in control of mosquito breeding sites'
outcome = 'History of malaria in the past year in the household'

# Control variable sets
control_vars_full = [
    ['Age'],
    ['Age', 'Gender'],
    ['Age', 'Gender', 'Level of education'],
    ['Age', 'Gender', 'Level of education', 'Marital status'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence'],
    ['Age', 'Gender', 'Level of education', 'Marital status', 'Occupation', 'Household size', 'Length of stay in area of residence', 'Area of residence']
]

# For rural/urban: drop area control
control_vars_split = [vars[:-1] for vars in control_vars_full if len(vars) == 8] + control_vars_full[:7]

# Hierarchical regression function
def hierarchical_logistic(area_data, area_name, control_vars):
    print(f"\n=== {area_name} Logistic Regression (Hierarchical) ===")
    for i, controls in enumerate(control_vars):
        model_vars = [predictor] + controls
        X = area_data[model_vars]
        y = area_data[outcome]
        print(f"\n--- Model {i+1}: Controls: {', '.join(controls)} ---")
        try:
            result = run_logistic(X, y)
            print(result.summary())
            odds_ratios = pd.DataFrame({
                'OR': np.exp(result.params),
                '2.5%': np.exp(result.conf_int()[0]),
                '97.5%': np.exp(result.conf_int()[1])
            })
            print("\nOdds Ratios with 95% CI:")
            print(odds_ratios)
        except Exception as e:
            print(f"Model {i+1} failed: {e}")

# Run regression for full dataset
hierarchical_logistic(data, "Full Sample", control_vars_full)

# Run regression for rural and urban
hierarchical_logistic(rural_data, "Rural Area", control_vars_split)
hierarchical_logistic(urban_data, "Urban Area", control_vars_split)


=== Full Sample Logistic Regression (Hierarchical) ===

--- Model 1: Controls: Age ---
                                            Logit Regression Results                                            
Dep. Variable:     History of malaria in the past year in the household   No. Observations:                  282
Model:                                                            Logit   Df Residuals:                      279
Method:                                                             MLE   Df Model:                            2
Date:                                                  Tue, 20 May 2025   Pseudo R-squ.:                0.006684
Time:                                                          09:30:22   Log-Likelihood:                -185.94
converged:                                                         True   LL-Null:                       -187.19
Covariance Type:                                              nonrobust   LLR p-value:                    0.2862
        